In [41]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from mcda.electre.concordance import *
from mcda.electre.discordance import *
from mcda.electre.credibility import *
from mcda.electre.outranking import *
from mcda.electre.outranking.choice import *
from mcda.electre.outranking.ranking import *
from mcda.electre.outranking.sorting import *
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold
from mcda.electre.plot import *

In [42]:
from operator import index


Audi   = [160, 20000, 700, 150]
Bmw = [200, 50000, 400, 180]
Fiat   = [170, 30000, 600, 100]
Honda = [180, 40000, 500, 160]
Opel   = [140, 35000, 800, 140]
cars = pd.DataFrame([Audi, Bmw, Fiat,  Honda,  Opel], index=['Audi', 'BMW' , 'Fiat' , 'Honda' , 'Opel'])

scales  = pd.Series([QuantitativeScale(0, 24000),
           QuantitativeScale(1, 100000000, PreferenceDirection.MIN), 
           QuantitativeScale(0, 10000000),
           QuantitativeScale(1, 2000000000000)])
weights = pd.Series([2, 3, 1, 2])
cars

,0,1,2,3
Audi,160,20000,700,150
BMW,200,50000,400,180
Fiat,170,30000,600,100
Honda,180,40000,500,160
Opel,140,35000,800,140


In [43]:
indifference = pd.Series([
    Threshold(0, 10),
    Threshold(0, 10000),
    Threshold(0, 100),
    Threshold(0, 10)
])
preference = pd.Series([
    Threshold(0, 20),
    Threshold(0, 18000),
    Threshold(0, 200),
    Threshold(0, 25)
])

pre_veto = pd.Series([
    Threshold(0, 40), 
    Threshold(0, 80000),
    Threshold(0, 400),
    Threshold(0, 40)
])

veto = pd.Series([
    Threshold(0, 60), 
    Threshold(0, 100000),
    Threshold(0, 500),
    Threshold(0, 50)
])

cv = pd.Series([
    Threshold(0, 60), 
    Threshold(0, 100000),
    None,
    Threshold(0, 50)
])

reinforce = pd.Series([
    Threshold(0, 50), 
    Threshold(0, 20000),
    None,
    Threshold(0, 40)
])

reinforce_factor = pd.Series([
    2.0, 
    1.8,
    None,
    1.5
])


In [44]:
concordance_comp = concordance(
    cars, scales, weights, indifference, preference
)
concordance_comp

,Audi,BMW,Fiat,Honda,Opel
Audi,1.000000,0.500000,1.00,0.750000,1.000000
BMW,0.500000,1.000000,0.50,1.000000,0.640625
Fiat,0.750000,0.500000,1.00,0.750000,0.625000
Honda,0.500000,0.583333,1.00,1.000000,0.875000
Opel,0.515625,0.500000,0.75,0.583333,1.000000


In [45]:
concordance_reinf = concordance_reinforced(
    cars, scales, weights, indifference, preference, reinforce, reinforce_factor
)
concordance_reinf

,Audi,BMW,Fiat,Honda,Opel
Audi,1.000000,0.615385,1.000000,0.750000,1.0000
BMW,0.500000,1.000000,0.555556,1.000000,0.7125
Fiat,0.750000,0.500000,1.000000,0.750000,0.6250
Honda,0.500000,0.583333,1.000000,1.000000,0.8750
Opel,0.515625,0.500000,0.750000,0.583333,1.0000


In [46]:
interactions = pd.DataFrame([
    [None, None, None, None],
    [Interaction(InteractionType.A, 1.4), None, None, None],
    [None, None, None, Interaction(InteractionType.MS, 3.5)],
    [None, Interaction(InteractionType.MW, -0.6), None, None],
])
interactions

,0,1,2,3
0,None,None,None,None
1,Antagonistic | 1.4,None,None,None
2,None,None,None,Mutual Strengthening | 3.5
3,None,Mutual Weakening | -0.6,None,None


In [47]:
concordance_with_int = concordance_with_interactions(cars, scales, weights, indifference, preference, interactions)
concordance_with_int.round(3)

,Audi,BMW,Fiat,Honda,Opel
Audi,1.000,0.394,1.000,0.789,1.000
BMW,0.500,1.000,0.500,1.000,0.630
Fiat,0.697,0.394,1.000,0.697,0.625
Honda,0.500,0.559,1.000,1.000,0.865
Opel,0.640,0.394,0.789,0.559,1.000


In [48]:
discordance_bi = discordance_bin(
    cars, scales, veto
)
discordance_bi

,Audi,BMW,Fiat,Honda,Opel
Audi,0,0,0,0,0
BMW,0,0,0,0,0
Fiat,1,1,0,1,0
Honda,0,0,0,0,0
Opel,0,1,0,0,0


In [49]:
marginals = discordance_marginals(cars, scales, preference, veto, pre_veto)
marginals.round(3)

,Audi,BMW,Fiat,Honda,Opel
Audi,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64
BMW,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64
Fiat,0 0.0 1 0.0 2 0.0 3 1.0 dtype: flo...,0 0 1 0 2 0 3 1 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 1 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64
Honda,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64
Opel,0 0 1 0 2 0 3 0 dtype: int64,0 1.0 1 0.0 2 0.0 3 0.0 dtype: flo...,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64,0 0 1 0 2 0 3 0 dtype: int64


In [50]:
discordance_comp = discordance(marginals, weights)
discordance_comp

,Audi,BMW,Fiat,Honda,Opel
Audi,0.00,0.00,0.0,0.00,0.0
BMW,0.00,0.00,0.0,0.00,0.0
Fiat,0.25,0.25,0.0,0.25,0.0
Honda,0.00,0.00,0.0,0.00,0.0
Opel,0.00,0.25,0.0,0.00,0.0


In [51]:
non_discor = non_discordance(marginals, NonDiscordanceType.DC, concordance_comp)
non_discor

,Audi,BMW,Fiat,Honda,Opel
Audi,1.0,1.0,1.0,1.0,1.0
BMW,1.0,1.0,1.0,1.0,1.0
Fiat,0.0,0.0,1.0,0.0,1.0
Honda,1.0,1.0,1.0,1.0,1.0
Opel,1.0,0.0,1.0,1.0,1.0


In [52]:
counter_v = counter_veto(cars, scales, cv)
counter_v

,Audi,BMW,Fiat,Honda,Opel
Audi,[],[],[],[],[]
BMW,[],[],[3],[],[]
Fiat,[],[],[],[],[]
Honda,[],[],[3],[],[]
Opel,[],[],[],[],[]


In [53]:
credibility_comp = credibility_comprehensive(non_discor, concordance_comp)
credibility_comp

,Audi,BMW,Fiat,Honda,Opel
Audi,1.000000,0.500000,1.00,0.750000,1.000000
BMW,0.500000,1.000000,0.50,1.000000,0.640625
Fiat,0.000000,0.000000,1.00,0.000000,0.625000
Honda,0.500000,0.583333,1.00,1.000000,0.875000
Opel,0.515625,0.000000,0.75,0.583333,1.000000


In [54]:
ccredibility_counter_v = credibility_cv(concordance_comp, non_discor, counter_v, 4)
ccredibility_counter_v

,Audi,BMW,Fiat,Honda,Opel
Audi,1.000000,0.500000,1.00,0.750000,1.000000
BMW,0.500000,1.000000,0.50,1.000000,0.640625
Fiat,0.000000,0.000000,1.00,0.000000,0.625000
Honda,0.500000,0.583333,1.00,1.000000,0.875000
Opel,0.515625,0.000000,0.75,0.583333,1.000000


In [55]:
credibility_electre_4 = credibility_electre_iv(cars, scales, indifference, preference, veto)
credibility_electre_4

,Audi,BMW,Fiat,Honda,Opel
Audi,1.0,0.0,1.0,0.2,1.0
BMW,0.0,1.0,0.0,0.6,0.2
Fiat,0.0,0.0,1.0,0.0,0.0
Honda,0.0,0.0,0.6,1.0,0.0
Opel,0.0,0.0,0.2,0.0,1.0


In [56]:
crisp_table = crisp_cut(credibility_comp, 0.7)
crisp_table

,Audi,BMW,Fiat,Honda,Opel
Audi,True,False,True,True,True
BMW,False,True,False,True,False
Fiat,False,False,True,False,False
Honda,False,False,True,True,True
Opel,False,False,True,False,True


In [57]:
outranking_rel = outranking_relation(crisp_table)
outranking_rel

,Audi,BMW,Fiat,Honda,Opel
Audi,OutrankingRelation.INDIFF,OutrankingRelation.R,OutrankingRelation.PQ,OutrankingRelation.PQ,OutrankingRelation.PQ
BMW,OutrankingRelation.R,OutrankingRelation.INDIFF,OutrankingRelation.R,OutrankingRelation.PQ,OutrankingRelation.R
Fiat,None,OutrankingRelation.R,OutrankingRelation.INDIFF,None,None
Honda,None,None,OutrankingRelation.PQ,OutrankingRelation.INDIFF,OutrankingRelation.PQ
Opel,None,OutrankingRelation.R,OutrankingRelation.PQ,None,OutrankingRelation.INDIFF


In [58]:
kernel = find_kernel(crisp_table)
kernel

['Audi', 'BMW']

In [59]:
nfs = net_flow_score(crisp_table)
nfs

Audi     3
BMW      1
Honda    0
Opel    -1
Fiat    -3
dtype: int64

In [60]:
down = distillation(credibility_comp)
down

1          [Audi]
2           [BMW]
3         [Honda]
4    [Fiat, Opel]
dtype: object

In [61]:
up = distillation(credibility_comp, upward_order=True)
up

3     [Audi, BMW]
2         [Honda]
1    [Fiat, Opel]
dtype: object

In [62]:
desc = order_to_outranking_matrix(down)
desc

,Audi,BMW,Honda,Fiat,Opel
Audi,1,1,1,1,1
BMW,0,1,1,1,1
Honda,0,0,1,1,1
Fiat,0,0,0,1,1
Opel,0,0,0,1,1


In [63]:
asc = order_to_outranking_matrix(up)
asc

,Audi,BMW,Honda,Fiat,Opel
Audi,1,1,1,1,1
BMW,1,1,1,1,1
Honda,0,0,1,1,1
Fiat,0,0,0,1,1
Opel,0,0,0,1,1


In [64]:
final = final_ranking_matrix(down, up)
final

,Audi,BMW,Honda,Fiat,Opel
Audi,1,1,1,1,1
BMW,0,1,1,1,1
Honda,0,0,1,1,1
Fiat,0,0,0,1,1
Opel,0,0,0,1,1


In [65]:
ranking = ranks(final)
ranking

1          [Audi]
2           [BMW]
3         [Honda]
4    [Fiat, Opel]
dtype: object

In [66]:
median = median_order(ranking, down, up)
median

1          [Audi]
2           [BMW]
3         [Honda]
4    [Fiat, Opel]
dtype: object

In [67]:
Good   = [180, 20000, 750, 170]
Good2   = [190, 25000, 700, 180]
Medium = [170, 35000, 650, 140]
Medium2 = [160, 300000, 550, 150]
Bad   = [145, 45000, 450, 120]
Bad2   = [130, 50000, 500, 130]
categories = pd.DataFrame([Bad, Bad2, Medium, Medium2, Good, Good2], index=["C1", "C1.2", "C2", "C2.2", "C3", "C3.2"])
categories

,0,1,2,3
C1,145,45000,450,120
C1.2,130,50000,500,130
C2,170,35000,650,140
C2.2,160,300000,550,150
C3,180,20000,750,170
C3.2,190,25000,700,180


In [68]:
concordance_ap, concordance_pa  = concordance(cars, scales, weights, indifference, preference, categories)

In [69]:
marginals_ap, marginals_pa = discordance_marginals(cars, scales, preference, veto, pre_veto, categories)

In [70]:
non_discordance_ap = non_discordance(marginals_ap, NonDiscordanceType.D)
non_discordance_pa = non_discordance(marginals_pa, NonDiscordanceType.D)

In [71]:
credibility_ap = credibility_comprehensive(non_discordance_ap, concordance_ap).round(3)
credibility_ap

,C1,C1.2,C2,C2.2,C3,C3.2
Audi,1.000,1.00,1.000,1.000,0.583,0.500
BMW,1.000,1.00,0.641,0.938,0.500,0.500
Fiat,0.833,0.75,0.750,0.000,0.000,0.000
Honda,1.000,1.00,0.938,1.000,0.500,0.474
Opel,1.000,1.00,0.750,0.750,0.266,0.250


In [72]:
credibility_pa = credibility_comprehensive(non_discordance_pa, concordance_pa).round(3)
credibility_pa

,Audi,BMW,Fiat,Honda,Opel
C1,0.125,0.00,0.453,0.500,0.708
C1.2,0.083,0.00,0.375,0.250,0.641
C2,0.766,0.50,1.000,0.833,0.938
C2.2,0.000,0.00,0.000,0.000,0.000
C3,1.000,0.75,1.000,1.000,1.000
C3.2,1.000,1.00,1.000,1.000,1.000


In [73]:
crisp_ap = crisp_cut(credibility_ap, 0.7)
crisp_ap

,C1,C1.2,C2,C2.2,C3,C3.2
Audi,True,True,True,True,False,False
BMW,True,True,False,True,False,False
Fiat,True,True,True,False,False,False
Honda,True,True,True,True,False,False
Opel,True,True,True,True,False,False


In [74]:
crisp_pa = crisp_cut(credibility_pa, 0.7)
crisp_pa

,Audi,BMW,Fiat,Honda,Opel
C1,False,False,False,False,True
C1.2,False,False,False,False,False
C2,True,False,True,True,True
C2.2,False,False,False,False,False
C3,True,True,True,True,True
C3.2,True,True,True,True,True


In [75]:
Tri_B = pd.Series(["C1", "C2", None], index = ["Bad", "Medium", "Good"])
Tri_nB = pd.Series([["C2", "C2.2"], ["C3", "C2.2"], []],  index = ["Bad", "Medium", "Good"])

In [76]:
assign_tri_b_class(crisp_ap, crisp_pa, Tri_B)

Audi       (Good, Good)
BMW      (Medium, Good)
Fiat       (Good, Good)
Honda      (Good, Good)
Opel       (Good, Good)
dtype: object

In [77]:
assign_tri_nb_class(crisp_ap, crisp_pa,  Tri_nB)

Audi       (Medium, Good)
BMW        (Medium, Good)
Fiat     (Medium, Medium)
Honda      (Medium, Good)
Opel       (Medium, Good)
dtype: object

In [78]:
Tri_C = pd.Series(["C1", "C2", "C3"], index = ["Bad", "Medium", "Good"])
assign_tri_c_class(crisp_ap, crisp_pa, credibility_ap, credibility_pa, Tri_C)

Audi     (Medium, Medium)
BMW      (Medium, Medium)
Fiat     (Medium, Medium)
Honda    (Medium, Medium)
Opel        (Bad, Medium)
dtype: object

In [79]:
Tri_nC = pd.Series([ ["C1", "C1.2"], ["C2", "C2.2"], ["C3", "C3.2"]], index = ["Bad", "Medium", "Good"])
assign_tri_nc_class(credibility_ap, credibility_pa, Tri_nC)

Audi     (Medium, Medium)
BMW        (Medium, Good)
Fiat     (Medium, Medium)
Honda    (Medium, Medium)
Opel        (Bad, Medium)
dtype: object

In [80]:
assign_tri_rc_class(crisp_ap, crisp_pa ,credibility_ap, credibility_pa, Tri_C)

Audi     (Medium, Medium)
BMW        (Medium, Good)
Fiat     (Medium, Medium)
Honda    (Medium, Medium)
Opel        (Bad, Medium)
dtype: object